# 02 - AlphaEarth Embeddings Exploration

Explore the 64-dimensional satellite embeddings from AlphaEarth.

Dataset: `GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL`

In [ ]:
import ee
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Initialize GEE
GCP_PROJECT_ID = 'gen-lang-client-0669818939'
ee.Initialize(project=GCP_PROJECT_ID)

In [ ]:
# Constants
ALPHAEARTH_ID = 'GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL'
BANDS = [f'A{i:02d}' for i in range(64)]  # A00 to A63

# Delhi NCR bounds
DELHI_BOUNDS = [76.8, 28.4, 77.4, 28.9]
delhi_region = ee.Geometry.Rectangle(DELHI_BOUNDS)

# Test points in Delhi
TEST_POINTS = {
    'Connaught Place': (28.6315, 77.2167),
    'India Gate': (28.6129, 77.2295),
    'Yamuna River': (28.6369, 77.2900),
    'IGI Airport': (28.5562, 77.1000)
}

## 1. Dataset Overview

In [ ]:
# Get collection info
collection = ee.ImageCollection(ALPHAEARTH_ID)

# Check available years
dates = collection.aggregate_array('system:time_start').getInfo()
years = sorted(set([ee.Date(d).format('YYYY').getInfo() for d in dates[:10]]))
print(f'Available years (sample): {years}')

# Get 2023 image
image_2023 = collection.filterDate('2023-01-01', '2023-12-31').first()
print(f'\nBands: {image_2023.bandNames().getInfo()[:10]}...')
print(f'Total bands: {len(BANDS)}')

## 2. Sample Embeddings at Points

In [ ]:
def get_embedding_at_point(lat, lng, year=2023):
    """Get 64-dim embedding for a point."""
    point = ee.Geometry.Point([lng, lat])
    image = collection.filterDate(f'{year}-01-01', f'{year}-12-31').first()
    
    sample = image.select(BANDS).sample(
        region=point,
        scale=10,
        numPixels=1
    ).first()
    
    if sample is None:
        return None
    
    props = sample.getInfo()['properties']
    return np.array([props.get(b, 0) for b in BANDS])

# Get embeddings for test points
embeddings = {}
for name, (lat, lng) in TEST_POINTS.items():
    print(f'Fetching embedding for {name}...')
    emb = get_embedding_at_point(lat, lng)
    if emb is not None:
        embeddings[name] = emb
        print(f'  Shape: {emb.shape}, Mean: {emb.mean():.4f}')

## 3. Visualize Embeddings

In [ ]:
# Plot embeddings as heatmap
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

for ax, (name, emb) in zip(axes.flat, embeddings.items()):
    ax.bar(range(64), emb)
    ax.set_title(f'{name}')
    ax.set_xlabel('Embedding Dimension')
    ax.set_ylabel('Value')
    ax.set_xlim(0, 63)

plt.tight_layout()
plt.savefig('alphaearth_embeddings.png', dpi=150)
plt.show()

## 4. Compare Similarity

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Create similarity matrix
names = list(embeddings.keys())
emb_matrix = np.array([embeddings[n] for n in names])

similarity = cosine_similarity(emb_matrix)

# Plot
plt.figure(figsize=(8, 6))
sns.heatmap(similarity, annot=True, fmt='.2f', 
            xticklabels=names, yticklabels=names,
            cmap='coolwarm', center=0.5)
plt.title('AlphaEarth Embedding Similarity (Cosine)')
plt.tight_layout()
plt.savefig('alphaearth_similarity.png', dpi=150)
plt.show()

## 5. Regional Statistics

In [ ]:
# Get mean embedding for entire Delhi region
image = collection.filterDate('2023-01-01', '2023-12-31').first().select(BANDS)

regional_stats = image.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=delhi_region,
    scale=100,  # 100m for speed
    maxPixels=1e9
).getInfo()

regional_mean = np.array([regional_stats.get(b, 0) for b in BANDS])

plt.figure(figsize=(12, 4))
plt.bar(range(64), regional_mean)
plt.title('Delhi NCR - Mean AlphaEarth Embedding')
plt.xlabel('Dimension')
plt.ylabel('Mean Value')
plt.tight_layout()
plt.savefig('delhi_mean_embedding.png', dpi=150)
plt.show()

print(f'Regional mean shape: {regional_mean.shape}')
print(f'Value range: [{regional_mean.min():.4f}, {regional_mean.max():.4f}]')

## Summary

- AlphaEarth provides 64-dimensional embeddings at 10m resolution
- Different land types (urban, river, airport) have distinct embedding patterns
- Embeddings can be used as spatial features for flood prediction